## 학습

### 모듈 가져오기

In [1]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

### 학습 데이터가 저장된 디렉토리를 데이터셋으로 설정

In [2]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

## 데이터를 학습 데이터와 검증 데이터(10개)로 분리

In [3]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 10, 10])

## 학습 데이터 로더 생성

In [4]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

## 사전 학습된 모델 로드(Alexnet)

In [5]:
model = models.alexnet(pretrained=True)
print(model)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /home/jetson/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth



AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)


## 전이 학습을 위해 FC의 마지막 레이어인 6번 레이어 수정
* Alexnet은 Imagenet 데이터베이스를 기반으로 학습되었으므로 카테고리가 1000개
* 본 예제의 카테고리는 blocked, free 두 개

In [6]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

## 생성된 모델을 GPU 영역으로 이동

In [8]:
device = torch.device('cuda')
model = model.to(device)

## 학습

In [9]:
NUM_EPOCHS = 15

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))

0: 1.000000
1: 1.000000
2: 1.000000
3: 1.000000
4: 1.000000
5: 1.000000
6: 1.000000
7: 1.000000
8: 1.000000
9: 1.000000
10: 1.000000
11: 1.000000
12: 1.000000
13: 1.000000
14: 1.000000


## 예측

### 모듈 가져오기

In [10]:
import cv2
import numpy as np

from jetbotmini import Camera, bgr8_to_jpeg
from jetbotmini import Robot

import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets

### 예측할 이미지 전처리
* 학습 이미지와 동일한 전처리가 필요함

In [11]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

### 카메라와 이미지 위젯 설정

In [12]:
camera = Camera.instance(width=224, height=224,fps=10)
image = widgets.Image(format='jpeg', width=224, height=224)

### 카메라와 이미지 위젯 연결

In [13]:
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

### 위젯 출력

In [14]:
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

### 모터 구동을 위한 로봇 객체 생성

In [15]:
robot = Robot()

### 장애물 예측 및 모터 구동

In [16]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    if prob_blocked < 0.7:
        robot.forward(0.6)
    else:
        robot.left(0.6)
    #if prob_blocked < 0.78:
        #robot.stop()
        #robot.forward(0.7)
    #else:
        #robot.stop()
        #robot.left(0.7)
    
    #time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize

In [17]:
camera.observe(update, names='value')

### 카메라 감지 중단

In [18]:
camera.unobserve(update, names='value')
time.sleep(1)
robot.stop()